<a href="https://colab.research.google.com/github/isharaU/EventDataGenerator/blob/main/Video_to_Event_data_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# check if the GPU is working properly
!nvidia-smi
%rm -rf ./sample_data

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.
UsageError: Line magic function `%rm` not found.


In [2]:
# Installing dependencies
%pip install numba -q
%pip install engineering-notation -q
%pip install opencv-contrib-python -q
%pip install argcomplete -q
%pip install dv-processing -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 680.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 25.1 MB/s eta 0:00:00


In [3]:
# Clone SensorsINI/v2e into colab
!git clone https://github.com/SensorsINI/v2e

Cloning into 'v2e'...
remote: Enumerating objects: 3314, done.
remote: Counting objects: 100% (1086/1086), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 3314 (delta 982), reused 1008 (delta 927), pack-reused 2228 (from 1)
Receiving objects: 100% (3314/3314), 34.34 MiB | 29.80 MiB/s, done.
Resolving deltas: 100% (2445/2445), done.


In [7]:
# install v2e
%cd /content/v2e
%pip install .

/content/v2e
Processing /content/v2e
  Preparing metadata (setup.py) ... done
  Created wheel for v2e: filename=v2e-1.5.1-py3-none-any.whl size=116981 sha256=8588e638133fcefc2c6112e71ff65d306d082d06c9578454248b529a731aee74
  Stored in directory: /tmp/pip-ephem-wheel-cache-mqy8eco1/wheels/77/5f/75/585e5c0362b6f0da630814aee1b3cce77ba29bfd5274e7fa38
Successfully built v2e
  Attempting uninstall: v2e
    Found existing installation: v2e 1.5.1
    Uninstalling v2e-1.5.1:
      Successfully uninstalled v2e-1.5.1
mkdir: cannot create directory ‘input’: File exists


In [8]:
# SuperSloMo39 will be saved here
!mkdir input

# go back
%cd /content/
%pwd

/content


'/content'

In [9]:
# bypass the google large file download confrimation
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx" -O /content/v2e/input/SuperSloMo39.ckpt && rm -rf /tmp/cookies.txt

--2024-09-20 14:37:36--  https://docs.google.com/uc?export=download&confirm=&id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx
Resolving docs.google.com (docs.google.com)... 173.194.194.102, 173.194.194.139, 173.194.194.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.194.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx&export=download [following]
--2024-09-20 14:37:36--  https://drive.usercontent.google.com/download?id=1ETID_4xqLpRBrRo1aOT7Yphs3QqWR_fx&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.125.132, 2607:f8b0:4001:c2f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.125.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2429 (2.4K) [text/html]
Saving to: ‘/content/v2e/input/SuperSloMo39.ckpt’

/content/v2e/input/ 100%[===================>]   2

In [10]:
# mount with goole drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#@title Paste the Video File Path Here: { run: "auto", vertical-output: true }
video_path = "/content/drive/MyDrive/Videos/WP_20170428_15_52_44_Pro.mp4" #@param {type:"string"}

import os
if video_path != "" and os.path.isfile(video_path):
    display("The chosen video file: {}".format(video_path))
else:
    print("The file path is empty or invalid, choose a file")


'The chosen video file: /content/drive/MyDrive/Videos/WP_20170428_15_52_44_Pro.mp4'

In [19]:
#@title **Configure V2E Options** { run: "auto", vertical-output: true, display-mode: "both" }
#@markdown The full documentation is here: https://github.com/SensorsINI/v2e#render-emulated-dvs-events-from-conventional-video

#@markdown This GUI supports only common use configs, for finer control, please check the v2e documentation and edit the code below.

#@markdown ---
#@markdown ## Output Options

output_folder = "/content/drive/MyDrive/Videos" #@param {type:"string"}
#@markdown - folder to store outputs. (default: v2e-output)
overwrite = True #@param {type:"boolean"}
#@markdown - overwrites files in existing folder (checks existence of non-empty output_folder). (default: True)
unique_output_folder = False #@param {type:"boolean"}
#@markdown - makes unique output folder based on output_folder, e.g. output1 (if non-empty output_folder already exists) (default: False)
out_filename = "events.h5" #@param {type:"string"}
#@markdown - Output DVS events as hdf5 event database.
davis_output = True #@param {type:"boolean"}
#@markdown - If also save frames in HDF5. (default: False)

#@markdown ### Output DVS Video Options
skip_video_output = False #@param {type:"boolean"}
#@markdown - Skip producing video outputs, including the original video, SloMo video, and DVS video (default: False)
dvs_exposure = "duration .033" #@param {type:"string"}
#@markdown - Mode to finish DVS frame event integration: duration time: Use fixed accumulation time in seconds, e.g. --dvs_exposure duration .005; count n: Count n events per frame, -dvs_exposure count 5000; area_event N M: frame ends when any area of M x M pixels fills with N events, -dvs_exposure area_count 500 64 (default: duration 0.01)
output_mode = "dvs346" #@param ["dvs128", "dvs240", "dvs346", "dvs640", "dvs1024"]
#@markdown - This option sets the output size of the v2e. Supported models: "dvs128" (128x128), "dvs240" (240x180), "dvs346" (346x260), "dvs640" (640x480), "dvs1024" (1024x768).

#@markdown ---

#@markdown ## Input Options
input_frame_rate = 30 #@param {type:"number"}
#@markdown - Manually define the video frame rate when the video is presented as a list of image files. When the input video is a video file, this option will be ignored.
input_slowmotion_factor =  1#@param {type:"number"}
#@markdown - Sets the known slow-motion factor of the input video, i.e. how much the video is slowed down, i.e., the ratio of shooting frame rate to playback frame rate. input_slowmotion_factor<1 for sped-up video and input_slowmotion_factor>1 for slowmotion video.If an input video is shot at 120fps yet is presented as a 30fps video (has specified playback frame rate of 30Hz, according to file's FPS setting), then set --input_slowdown_factor=4.It means that each input frame represents (1/30)/4 s=(1/120)s.If input is video with intended frame intervals of 1ms that is in AVI file with default 30 FPS playback spec, then use ((1/30)s)*(1000Hz)=33.33333. (default: 1.0)

#@markdown ---

#@markdown ## DVS Time Resolution Options
disable_slomo = True #@param {type:"boolean"}
#@markdown - Disables slomo interpolation; the output DVS events will have exactly the timestamp resolution of the source video (which is perhaps modified by --input_slowmotion_factor). (default: False)
timestamp_resolution = 0.001 #@param {type:"number"}
#@markdown - (Ignored by --disable_slomo.) Desired DVS timestamp resolution in seconds; determines slow motion upsampling factor; the video will be upsampled from source fps to achieve the at least this timestamp resolution.I.e. slowdown_factor = (1/fps)/timestamp_resolution; using a high resolution e.g. of 1ms will result in slow rendering since it will force high upsampling ratio. Can be combind with --auto_timestamp_resolution to limit upsampling to a maximum limit value. (default: None)
auto_timestamp_resolution = True #@param {type:"boolean"}
#@markdown - (Ignored by --disable_slomo.) If True (default), upsampling_factor is automatically determined to limit maximum movement between frames to 1 pixel. If False, --timestamp_resolution sets the upsampling factor for input video. Can be combined with --timestamp_resolution to ensure DVS events have at most some resolution. (default: False)

# This is the SloMo model path
slomo_model = "/content/v2e/input/SuperSloMo39.ckpt"

#@markdown ---

#@markdown ## DVS Model Options
condition = "Noisy" #@param ["Custom", "Clean", "Noisy"]
#@markdown - Custom: Use following slidebar to adjust your DVS model.
#@markdown - Clean: a preset DVS model, generate clean events, without non-idealities.
#@markdown - Noisy: a preset DVS model, generate noisy events.

thres = 0.2 #@param {type:"slider", min:0.05, max:1, step:0.01}
#@markdown - threshold in log_e intensity change to trigger a positive/negative event. (default: 0.2)
sigma = 0.03 #@param {type:"slider", min:0.01, max:0.25, step:0.001}
#@markdown - 1-std deviation threshold variation in log_e intensity change. (default: 0.03)
cutoff_hz = 200 #@param {type:"slider", min:0, max:300, step:1}
#@markdown - photoreceptor first-order IIR lowpass filter cutoff-off 3dB frequency in Hz - see https://ieeexplore.ieee.org/document/4444573 (default: 300)
leak_rate_hz = 5.18 #@param {type:"slider", min:0, max:100, step:0.01}
#@markdown - leak event rate per pixel in Hz - see https://ieeexplore.ieee.org/abstract/document/7962235 (default: 0.01)
shot_noise_rate_hz = 2.716 #@param {type:"slider", min:0, max:100, step:0.001}
#@markdown - Temporal noise rate of ON+OFF events in darkest parts of scene; reduced in brightest parts. (default: 0.001)

if condition == "Clean":
    thres = 0.2
    sigma = 0.02
    cutoff_hz = 0
    leak_rate_hz = 0
    shot_noise_rate_hz = 0
elif condition == "Noisy":
    thres = 0.2
    sigma_thres = 0.05
    cutoff_hz = 30
    leak_rate_hz = 0.1
    shot_noise_rate_hz = 5

#@markdown ---

v2e_command = ["v2e"]

# set the input folder
# the video_path can be a video file or a folder of images
v2e_command += ["-i", video_path]

# set the output folder
v2e_command += ["-o", output_folder]

# if the output will rewrite the previous output
if overwrite:
    v2e_command.append("--overwrite")

# if there the output folder is unique
v2e_command += ["--unique_output_folder", "{}".format(unique_output_folder).lower()]

# set output configs, for the sake of this tutorial, let's just output HDF5 record
if davis_output:
    v2e_command += ["--davis_output"]
v2e_command += ["--dvs_h5", out_filename]
v2e_command += ["--dvs_aedat2", "None"]
v2e_command += ["--dvs_text", "None"]

# in Colab, let's say no preview
v2e_command += ["--no_preview"]

# if skip video output
if skip_video_output:
    v2e_command += ["--skip_video_output"]
else:
    # set DVS video rendering params
    v2e_command += ["--dvs_exposure", dvs_exposure]

# set slomo related options
v2e_command += ["--input_frame_rate", "{}".format(input_frame_rate)]
v2e_command += ["--input_slowmotion_factor", "{}".format(input_slowmotion_factor)]

# set slomo data
if disable_slomo:
    v2e_command += ["--disable_slomo"]
    v2e_command += ["--auto_timestamp_resolution", "false"]
else:
    v2e_command += ["--slomo_model", slomo_model]
    if auto_timestamp_resolution:
        v2e_command += ["--auto_timestamp_resolution", "{}".format(auto_timestamp_resolution).lower()]
    else:
        v2e_command += ["--timestamp_resolution", "{}".format(timestamp_resolution)]

# threshold
v2e_command += ["--pos_thres", "{}".format(thres)]
v2e_command += ["--neg_thres", "{}".format(thres)]

# sigma
v2e_command += ["--sigma_thres", "{}".format(sigma)]

# DVS non-idealities
v2e_command += ["--cutoff_hz", "{}".format(cutoff_hz)]
v2e_command += ["--leak_rate_hz", "{}".format(leak_rate_hz)]
v2e_command += ["--shot_noise_rate_hz", "{}".format(shot_noise_rate_hz)]

# append output mode
v2e_command += [f"--{output_mode}"]

# Final v2e command

final_v2e_command = " ".join(v2e_command)

print("The Final v2e command:")
display(final_v2e_command)


The Final v2e command:


'v2e -i /content/drive/MyDrive/Videos/WP_20170428_15_52_44_Pro.mp4 -o /content/drive/MyDrive/Videos --overwrite --unique_output_folder false --davis_output --dvs_h5 events.h5 --dvs_aedat2 None --dvs_text None --no_preview --dvs_exposure duration .033 --input_frame_rate 30 --input_slowmotion_factor 1 --disable_slomo --auto_timestamp_resolution false --pos_thres 0.2 --neg_thres 0.2 --sigma_thres 0.03 --cutoff_hz 30 --leak_rate_hz 0.1 --shot_noise_rate_hz 5 --dvs346'

In [20]:
# Run command!
!$final_v2e_command

INFO:v2e:torch device is cpu
INFO:v2e:No module named 'gooey': Gooey GUI builder not available, will use command line arguments.
Install with 'pip install Gooey if you want a no-arg GUI to invoke v2e'. See README
INFO:v2e:name 'Gooey' is not defined: Gooey package GUI not available, using command line arguments. 
You can try to install with "pip install Gooey"
INFO:v2ecore.v2e_utils:using output folder /content/drive/MyDrive/Videos
INFO:v2e:output_in_place==False so made output_folder=/content/drive/MyDrive/Videos
INFO:v2ecore.v2e_args:
*** arguments:
auto_timestamp_resolution:	False
avi_frame_rate:	30
batch_size:	8
crop:	None
cs_lambda_pixels:	None
cs_tau_p_ms:	None
cutoff_hz:	30.0
ddd_output:	False
disable_slomo:	True
dvs1024:	False
dvs128:	False
dvs240:	False
dvs346:	True
dvs640:	False
dvs_aedat2:	None
dvs_aedat4:	None
dvs_emulator_seed:	0
dvs_exposure:	['duration', '.033']
dvs_h5:	events.h5
dvs_params:	None
dvs_text:	None
dvs_vid:	dvs-video.avi
dvs_vid_full_scale:	2
hdr:	False
inpu

In [23]:
# Output file path
import os
import h5py
out_filepath = os.path.join(output_folder, out_filename)
out_dataset = h5py.File(out_filepath, "r")

file_stats = os.stat(out_filepath)

# Summarize file

events = out_dataset["events"]

num_events = events.shape[0]
duration = (events[-1, 0]-events[0, 0])/1e6  # in seconds
event_rate = num_events/duration/1000

display("File Size (MB): {:.2f}".format(file_stats.st_size / (1024*1024)))
display("Number of Events: {}".format(events.shape[0]))
display("Recording Duration: {:.2f}s".format(duration))
display("Event Rate: {:.2f} KEVs/s".format(event_rate))

'File Size (MB): 174.64'

'Number of Events: 39779174'

'Recording Duration: 8.03s'

'Event Rate: 4952.18 KEVs/s'

In [24]:
# DVS Video
dvs_vid_path = os.path.join(output_folder, "dvs-video.avi")
!ffmpeg -y -i $dvs_vid_path /tmp/output.mp4 -loglevel quiet

from IPython.display import HTML
from base64 import b64encode
mp4 = open("/tmp/output.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("<video controls src={} width=50%/>".format(data_url))